### **Part 4**
**Tasks**
* explore proper visual stimuli
  * spontaneous state
  * (done) evoked state: drifting
* Communication subspaces across different areas
  * (done) correlation
  * (done) dimensionality
    * intra_areas
    * inter_areas
* Information flow across subspaces
  * (done) direction: population correlation as the function of the time delay between areas ($t_2$-$t_1$)
  * intensity (correlation intensity)
* Hierarchy of information flow  across these subspaces
  * direction
    * (done) inter_areas cca_delay
    * *intra_areas cca_delay (layer_dependent)*
    * (done) feedfward and feedback
    * (done) input/output dimensions
  * function validation
* Relation between different information flow channels
  * *subspace & layer_dependent*
  * shared dimensionility
  * (done) generalize canonical dimension across subspaces
  * (done) subspaces angle
  * CCA across subspaces
  * predictive coding
* (to do) State dependent geometrical properties of these channels
* (to do) Dynamical system across areas modeled by multi-region RNN network
***************************************
(doing) 
* Layer dependent information flow
* Task8 Relation between area_subspace & layer_channel
**************************************

In [1]:
import os
import torch
import cupy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import subspace_angles
from sklearn.model_selection import KFold
from sklearn.decomposition import FactorAnalysis
from sklearn.cross_decomposition import CCA
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

### Task7 Layer dependent information flow

### Task8 Relation between area_subspace & layer_channel

***********************************************************************
Below Code copy from part4 for debug
***********************************************************************

In [2]:
basepath = "/home/jialab/Allensdk_data/local/ecephys_cache_dir/"
manifest_path = os.path.join(basepath, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
session = cache.get_session_data(755434585)
drift_stim_table = session.get_stimulus_table('drifting_gratings')
drift_stim_table.head()

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3798,0.8,225,"[21235.93333333, 21235.93333333]","[250.0, 250.0]",0.04,1585.891887,2,drifting_gratings,1587.893537,1,2.00165,246
3799,0.8,270,"[21235.93333333, 21235.93333333]","[250.0, 250.0]",0.04,1588.894403,2,drifting_gratings,1590.896063,2,2.00166,247
3800,0.8,0,"[21235.93333333, 21235.93333333]","[250.0, 250.0]",0.04,1591.896927,2,drifting_gratings,1593.898597,8,2.00167,248
3801,0.8,0,"[21235.93333333, 21235.93333333]","[250.0, 250.0]",0.04,1594.899423,2,drifting_gratings,1596.901083,2,2.00166,249
3802,0.8,315,"[21235.93333333, 21235.93333333]","[250.0, 250.0]",0.04,1597.901957,2,drifting_gratings,1599.903567,2,2.00161,250


In [3]:
def get_units_area(areas, session):

    units_area = np.zeros(len(areas))

    for i, area in enumerate(areas):
        units_area[i] = session.units[session.units["ecephys_structure_acronym"] == area].shape[0]

    return units_area

def spike_matrix(area, stim_table, bin=0.1, period=2):
    """spike_matrix, get spike_counts using function "presentationwise_spike_counts"

    Arguments:
        area -- brain area, which need to be analyzed
        stim_table -- stimulus_table got by allensdk, which need to be analyzed

    Keyword Arguments:
        bin -- count spikes within time bin, s (default: {0.1})
        period -- the whole time period of one stimuli in drift_grating_stimuli, s (default: {2})

    Returns:
        response_matrix, shape (stims, bins, units)
    """

    area_units = session.units[session.units["ecephys_structure_acronym"] == area]

    time_bins = np.arange(0, period + bin, bin)  

    spike_counts = session.presentationwise_spike_counts(
        stimulus_presentation_ids=stim_table.index.values,  
        bin_edges=time_bins,
        unit_ids=area_units.index.values
    )
    
    response_matrix = spike_counts.values

    return response_matrix

def get_design_matrix(area, stim_table):
    """get_design_matrix design_matrix for further analysis

    Reshape response_matrix from (stims, bins, units) to (stims*bins, units)/(n_samples, units)

    Arguments:
        area -- brain area, which need to be analyzed
        stim_table -- stimulus_table got by allensdk, which need to be analyzed

    Returns:
        design_matrix, shape (n_samples, units)
    """

    response_matrix = spike_matrix(area=area, stim_table=stim_table)
    design_matrix = response_matrix.reshape(response_matrix.shape[0]*response_matrix.shape[1],
                                    response_matrix.shape[2])

    return design_matrix

In [4]:
def cross_val_pCCA(des_mat, latent_dim):

    k_fold = 10
    kf = KFold(n_splits=k_fold)
    log_like = np.zeros(k_fold)

    fold = 0
    for train_index, test_index in kf.split(des_mat):
        train, test = des_mat[train_index], des_mat[test_index]
        # print("TRAIN:", train.shape, "TEST:", test.shape)

        fa = FactorAnalysis(n_components=latent_dim)
        fa.fit(train)

        log_like[fold] = fa.score(test)   # get Average log-likelihood of test
        fold = fold + 1

    return log_like.mean()

def inter_dim_pCCA(des_mat_1, des_mat_2):
    
    des_mat = np.concatenate((des_mat_1, des_mat_2), axis = 1)

    N, n_features = des_mat.shape
    _, n_features_1 = des_mat_1.shape
    _, n_features_2 = des_mat_2.shape

    # select proper latent dimensions from [1, n_features-1]
    cv_log_like = np.zeros(n_features-1)
    for i in range(n_features-1):
        latent_dim = i + 1
        cv_log_like[i] = cross_val_pCCA(des_mat, latent_dim)

    latent_dim_log_like = np.argmax(cv_log_like) + 1
    print('latent_dim_log_like', latent_dim_log_like)

    # select proper latent dimensions with best dim got by log_like
    fa = FactorAnalysis(n_components=latent_dim_log_like)
    fa.fit(des_mat)
    load_mat = fa.components_   # (n_components, n_features)
    load_mat_1 = load_mat[:, :n_features_1]
    load_mat_2 = load_mat[:, n_features_1:]

    _, sing_vals, _ = np.linalg.svd(load_mat_1.T @ load_mat_2)

    shared_var = np.cumsum(sing_vals)/np.sum(sing_vals)

    threshold = 0.9
    latent_dim = np.where(shared_var>threshold)[0][0]
    print('latent_dim', latent_dim)

    # use the proper latent_dim to fit
    fa = FactorAnalysis(n_components=latent_dim)
    fa.fit(des_mat)
    load_mat = fa.components_   # (n_components, n_features)
    load_mat_1 = load_mat[:, :n_features_1]
    load_mat_2 = load_mat[:, n_features_1:]

    return latent_dim, (load_mat_1, load_mat_2)

def get_inter_dim_area(areas, stim_table):

    inter_dim_areas = np.zeros([len(areas), len(areas)])

    for i, area_1 in enumerate(areas):
        des_mat_1 = get_design_matrix(area_1, stim_table)
        for j, area_2 in enumerate(areas):
            des_mat_2 = get_design_matrix(area_2, stim_table)
            inter_dim_areas[i, j], _ = inter_dim_pCCA(des_mat_1, des_mat_2)
            
    print('inter_dim_areas', inter_dim_areas)

    return inter_dim_areas

def plot_inter_dim_areas(areas, intra_dim_area, inter_dim_areas):
    # inter_dimension across every two areas

    f, axes = plt.subplots(nrows=len(areas), ncols=1, figsize=(24, 50), sharex=True)

    width=0.25
    x = np.array(range(len(areas)))
    intra_dim_area_2 = intra_dim_area

    for i, area_1 in enumerate(areas):

        intra_dim_area_1 = np.ones_like(x) * intra_dim_area[i]

        axes[i].bar(x, intra_dim_area_1, width=width, label='intra_dim_1', fc='r') 
        axes[i].bar(x+np.array([width]), inter_dim_areas[i, :], width=width, label='inter_dim', fc='g')
        axes[i].bar(x+np.array([2*width]), intra_dim_area_2, width=width, label='intra_dim_2', tick_label=areas, fc='b')
        axes[i].set_ylabel(f'{area_1}')
        axes[i].legend()

    f.suptitle('inter_dimension & intra_dimension across every two areas')
    plt.tight_layout()
    plt.show()

In [5]:
def pCCA_subspaces_angle(area_x, area_y1, area_y2, stim_table):
    """pCCA_subspaces_angle, get angles of subspaces via pCCA

    To identify the relation between subspaces got by (pCCA for area_x & area_y1) and (pCCA for area_x & area_y2)
    Use pCCA for area_x & area_y1, we can get information flow subspace (spanned by load_mat_xy1) of area_x and area_y1;
    Use pCCA for area_x & area_y2, we can get information flow subspace (spanned by load_mat_xy2) of area_x and area_y2;
    Then, by calculating the angle between these subspaces load_mat_xy1 & load_mat_xy2,
    we find that when area_x exchange information with area_y1 & area_y2, it use different subspaces.

    there are three relations of information flow between area_x, area_y1, area_y2:
    1. area_x  --->  area_y1 & area_x  --->  area_y2
    2. area_y1  --->  area_x & area_x  --->  area_y2
    3. area_y1  --->  area_x & area_y2  --->  area_x

    Arguments:
        area_x -- an area have information flow with both area_y1 & area_y2
        area_y1 -- an area have information flow with area_x
        area_y2 -- an area have information flow with area_x
        stim_table -- stimulus_table got by allensdk, which need to be analyzed

    Returns:
        subspaces_angle -- the angle of subspaces load_mat_xy1 & load_mat_xy2
    """

    des_mat_x = get_design_matrix(area_x, stim_table)
    des_mat_y1 = get_design_matrix(area_y1, stim_table)
    des_mat_y2 = get_design_matrix(area_y2, stim_table)

    _, (load_mat_xy1, _) = inter_dim_pCCA(des_mat_x, des_mat_y1)
    _, (load_mat_xy2, _) = inter_dim_pCCA(des_mat_x, des_mat_y2)
    print('load_mat_xy1', load_mat_xy1.shape)
    print('load_mat_xy2', load_mat_xy2.shape)

    subspaces_angle = np.rad2deg(subspace_angles(load_mat_xy1.T, load_mat_xy2.T))

    return subspaces_angle

def plot_subspaces_angles_area(area_1, area_2, areas_3, subspaces_angles):

    plt.figure(figsize=(16, 10))
    plt.plot(range(len(areas_3)), subspaces_angles)
    plt.xticks(ticks=range(len(areas_3)), labels=areas_3)
    plt.ylabel(f'subspaces_angles between {area_2} with')
    plt.title(f'subspaces_angles in {area_1}')
    plt.show() 

    return

In [6]:
area_1 = 'VISp'
area_2 = 'VISam'
areas_3 = ['VISp', 'VISrl', 'VISl', 'VISal', 'VISpm', 'VISam', 'LGd', 'LP']

subspaces_angles = np.zeros(len(areas_3))

for i, area_3 in enumerate(areas_3):
    subspaces_angles[i] = pCCA_subspaces_angle(area_1, area_2, area_3, stim_table=drift_stim_table)

np.save('subspaces_angles_755434585', subspaces_angles)
print('subspaces_angles', subspaces_angles)
plot_subspaces_angles_area(area_1, area_2, areas_3, subspaces_angles)